
Conclusion

Add in some of your own “multi-fields” to index the content in different ways using the Field Mapping settings
Index some different data types that we didn’t try out, such as latitude and longitude. How would you model searching what stores have what books in our data type?
Try out some more sophisticated queries that combine several different query types, filters and aggregations.

In [1]:
#%pip install opensearch-py -Uqq
# %pip install rich -Uqq

In [2]:
from fastcore.net import *
from fastcore.test import *
from nbdev.showdoc import show_doc
from rich import inspect as rinspect
from rich import print as rprint

In [3]:
es='http://localhost:9200'

urljson(es)

{'name': 'fedora',
 'cluster_name': 'opensearch',
 'cluster_uuid': 'QX9O66TzSZiJ8gNePiB63Q',
 'version': {'distribution': 'opensearch',
  'number': '1.2.4',
  'build_type': 'tar',
  'build_hash': 'e505b10357c03ae8d26d675172402f2f2144ef0f',
  'build_date': '2022-01-14T03:38:06.881862Z',
  'build_snapshot': False,
  'lucene_version': '8.10.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [4]:
urljson(f'{es}/_cluster/health')

{'cluster_name': 'opensearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'discovered_master': True,
 'active_primary_shards': 4,
 'active_shards': 4,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 3,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 57.14285714285714}

In [5]:
from opensearchpy import *

In [6]:
host, port = 'localhost', 9200
cli = OpenSearch([{'host': host, 'port': port}])

In [7]:
rinspect(cli.cat.health)

╭─ <bound method CatClient.health of <opensearchpy.client.cat.CatClient object at 0x7f05897─╮
│ def CatClient.health(params=None, headers=None):                                          │
│                                                                                           │
│ Returns a concise representation of the cluster health.                                   │
│                                                                                           │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                              │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

In [8]:
rinspect(cli.cat.indices)

╭─ <bound method CatClient.indices of <opensearchpy.client.cat.CatClient object at 0x7f0589─╮
│ def CatClient.indices(index=None, params=None, headers=None):                             │
│                                                                                           │
│ Returns information about indices: number of primaries and replicas, document             │
│ counts, disk size, ...                                                                    │
│                                                                                           │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                              │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
cli.cat.health()

'1644757436 13:03:56 opensearch yellow 1 1 true 4 4 0 0 3 0 - 57.1%\n'

In [10]:
rinspect(cli.cat.count)

╭─ <bound method CatClient.count of <opensearchpy.client.cat.CatClient object at 0x7f05897e─╮
│ def CatClient.count(index=None, params=None, headers=None):                               │
│                                                                                           │
│ Provides quick access to the document count of the entire cluster, or                     │
│ individual indices.                                                                       │
│                                                                                           │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                              │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
# Create an index with non-default settings.

index_name = 'searchml_revisited'
index_body = {
    'settings': {
        'index': {
            'query': {
                'default_field': "body"
            }
        }
    }
}
def _exists_index(index):
    return cli.cat.indices(index, h='index').strip() == index

if not _exists_index('searchml_revisited'):
    cli.indices.create(index_name, body=index_body)

In [12]:
test_eq(cli.cat.indices('searchml_revisited', h='index').strip(), 'searchml_revisited')

In [13]:
# Add our sample document to the index.
docs = [
    {
        "id": "doc_a",
        "title": "Fox and Hounds",
        "body": "The quick red fox jumped over the lazy brown dogs.",
        "price": "5.99",
        "in_stock": True,
        "category": "childrens"},
    {
        "id": "doc_b",
        "title": "Fox wins championship",
        "body": "Wearing all red, the Fox jumped out to a lead in the race over the Dog.",
        "price": "15.13",
        "in_stock": True,
        "category": "sports"},
    {
        "id": "doc_c",
        "title": "Lead Paint Removal",
        "body": "All lead must be removed from the brown and red paint.",
        "price": "150.21",
        "in_stock": False,
        "category": "instructional"},
    {
        "id": "doc_d",
        "title": "The Three Little Pigs Revisted",
        "price": "3.51",
        "in_stock": True,
        "body": "The big, bad wolf huffed and puffed and blew the house down. The end.",
        "category": "childrens"}
]

In [14]:
rinspect(cli.index)

╭─ <bound method OpenSearch.index of <OpenSearch([{'host': 'localhost', 'port': 9200}])>> ─╮
│ def OpenSearch.index(index, body, doc_type=None, id=None, params=None, headers=None):    │
│                                                                                          │
│ Creates or updates a document in an index.                                               │
│                                                                                          │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                             │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [16]:
index_name='searchml_revisited'
[cli.index(index_name, body=doc, id=doc['id'], refresh=True) for doc in docs]

[{'_index': 'searchml_revisited',
  '_type': '_doc',
  '_id': 'doc_a',
  '_version': 1,
  'result': 'created',
  'forced_refresh': True,
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 0,
  '_primary_term': 4},
 {'_index': 'searchml_revisited',
  '_type': '_doc',
  '_id': 'doc_b',
  '_version': 1,
  'result': 'created',
  'forced_refresh': True,
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 1,
  '_primary_term': 4},
 {'_index': 'searchml_revisited',
  '_type': '_doc',
  '_id': 'doc_c',
  '_version': 1,
  'result': 'created',
  'forced_refresh': True,
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 2,
  '_primary_term': 4},
 {'_index': 'searchml_revisited',
  '_type': '_doc',
  '_id': 'doc_d',
  '_version': 1,
  'result': 'created',
  'forced_refresh': True,
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 3,
  '_primary_term': 4}]